<a href="https://colab.research.google.com/github/Electromensch/nlp_need/blob/main/ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from bs4 import BeautifulSoup
import requests
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import pandas as pd

In [ ]:
url_kor = 'https://musulmanin.com/koran-na-russkom.html'
url_base = 'https://quran-online.ru/'
html_doc = requests.get(url_base).text
soup = BeautifulSoup(html_doc, 'html.parser')

In [ ]:
all_links2 = [re.findall(r'https://quran-online.ru/\d+/saadi', link.get('href')) for link in soup.find_all('a')]
all_links2 = [link[0] for link in all_links2 if link]

In [ ]:
mylinks = list(dict.fromkeys(all_links2))

In [ ]:
chunks = []
for link in range(len(mylinks)):
    html_doc2 = requests.get(mylinks[link]).text
    soup2 = BeautifulSoup(html_doc2, 'html.parser')
    need_text = soup2.find_all("div", attrs={'class': "ayat-text-type russian active", 'style': "display:block"})
    chunks += [i.getText().strip().replace('\n', ' ').replace(r'[!:.;?]\.', '.') for i in need_text]

In [ ]:
koran = ' '.join(chunks)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install ebooklib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 5.1 MB/s 
  Created wheel for ebooklib: filename=EbookLib-0.17.1-py3-none-any.whl size=38184 sha256=115b11424566b69d0b9d344435aa966638d7bd321445c2ce1b40100c01443560
  Stored in directory: /root/.cache/pip/wheels/43/39/fd/db4f652431a55d28472ba7f5f7c9a8efad03b97f443a48ea2f
Successfully built ebooklib


In [ ]:
import ebooklib
from ebooklib import epub

In [ ]:
!rm -r nlp_need
!git clone https://github.com/Electromensch/nlp_need

rm: cannot remove 'nlp_need': No such file or directory
Cloning into 'nlp_need'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
book = epub.read_epub('nlp_need/restruct.epub')
items = list(book.get_items_of_type(ebooklib.ITEM_DOCUMENT))

In [ ]:
texts = []

for c in items:
    soup3 = BeautifulSoup(c.get_body_content(), 'html.parser')
    text = [para.get_text() for para in soup3.find_all('p')]
    texts += text
    
tesak = ' '.join(texts)
tesak=tesak.replace('\\n', '')

Data Frame creation

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tesak_sent = sent_tokenize(tesak, language="russian")
koran_sent = sent_tokenize(koran, language="russian")

In [ ]:
df1 = pd.DataFrame(tesak_sent, columns=['sentences'])
df1['text'] = 'Tesak'
df1.head()
df2 = pd.DataFrame(koran_sent, columns=['sentences'])
df2['text'] = 'Koran'
df2.head()

,sentences,text
0,"Во имя Аллаха, Милостивого, Милосердного!",Koran
1,Это означает: я начинаю во имя Аллаха.,Koran
2,Из лексического анализа этого словосочетания я...,Koran
3,"Аллах - одно из этих имен, означающее «Бог, Ко...",Koran
4,Прекрасные имена Милостивый и Милосердный свид...,Koran


In [ ]:
df = pd.concat([df1, df2])
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

In [ ]:
df.head()

,sentences,text
0,"Вот принесли эти задания в колледж, надо решит...",Tesak
1,"Когда-то у меня по химии была пятерка, а я по ...",Tesak
2,"На зоне не нужно больше знать и уметь, кроме с...",Tesak
3,"Только я сказал, сразу меня начали просить по...",Tesak
4,"Мы садимся, я открываю эти задачи и понимаю, ч...",Tesak


In [ ]:
df.to_pickle('nlp_pickle')

In [ ]:
!pwd

/content


In [ ]:
from nltk.probability import FreqDist
import operator
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

In [ ]:
import spacy

In [ ]:
!python3 -m spacy download ru_core_news_md

In [ ]:
sp = spacy.load("ru_core_news_md")

In [25]:
df['tokens'] = df['sentences'].apply(lambda x: sp(x))

In [29]:
df.head()

,sentences,text,tokens,pos_tags
0,"Вот принесли эти задания в колледж, надо решит...",Tesak,"(Вот, принесли, эти, задания, в, колледж, ,, н...","{'вот': 'PART', 'принести': 'VERB', 'этот': 'D..."
1,"Когда-то у меня по химии была пятерка, а я по ...",Tesak,"(Когда, -, то, у, меня, по, химии, была, пятер...","{'когда': 'ADV', '-': 'ADV', 'то': 'ADV', 'у':..."
2,"На зоне не нужно больше знать и уметь, кроме с...",Tesak,"(На, зоне, не, нужно, больше, знать, и, уметь,...","{'на': 'ADP', 'зона': 'NOUN', 'не': 'PART', 'н..."
3,"Только я сказал, сразу меня начали просить по...",Tesak,"(Только, , я, сказал, ,, сразу, меня, начали,...","{'только': 'PART', ' ': 'SPACE', 'я': 'PRON', ..."
4,"Мы садимся, я открываю эти задачи и понимаю, ч...",Tesak,"(Мы, садимся, ,, я, открываю, эти, задачи, и, ...","{'мы': 'PRON', 'садиться': 'VERB', ',': 'PUNCT..."


In [28]:
df['pos_tags'] = df['tokens'].apply(lambda x: {i.lemma_: i.pos_ for i in x})

In [ ]:
df['ner'] = df['tokens'].apply(lambda x: )

In [8]:
df_train = df.head()

In [32]:
df_train['ner'] = df_train['tokens'].apply(lambda x: x.ents)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df = pd.read_pickle('mypkl')

In [33]:
df_train.head()

,sentences,text,tokens,pos_tags,ner
0,"Вот принесли эти задания в колледж, надо решит...",Tesak,"(Вот, принесли, эти, задания, в, колледж, ,, н...","{'вот': 'PART', 'принести': 'VERB', 'этот': 'D...",()
1,"Когда-то у меня по химии была пятерка, а я по ...",Tesak,"(Когда, -, то, у, меня, по, химии, была, пятер...","{'когда': 'ADV', '-': 'ADV', 'то': 'ADV', 'у':...",()
2,"На зоне не нужно больше знать и уметь, кроме с...",Tesak,"(На, зоне, не, нужно, больше, знать, и, уметь,...","{'на': 'ADP', 'зона': 'NOUN', 'не': 'PART', 'н...",()
3,"Только я сказал, сразу меня начали просить по...",Tesak,"(Только, , я, сказал, ,, сразу, меня, начали,...","{'только': 'PART', ' ': 'SPACE', 'я': 'PRON', ...",()
4,"Мы садимся, я открываю эти задачи и понимаю, ч...",Tesak,"(Мы, садимся, ,, я, открываю, эти, задачи, и, ...","{'мы': 'PRON', 'садиться': 'VERB', ',': 'PUNCT...",()


In [ ]:
!pip install -q deeppavlov

In [ ]:
!python -m deeppavlov install ner_rus
!python -m deeppavlov install ner_rus_bert

In [5]:
from deeppavlov import configs, build_model

config_path = configs.ner.ner_rus_bert


In [ ]:
ner = build_model(config_path, download=True)

In [18]:
def for_bert(text):
  tokens, tags = ner([text])
  return {tok: tag for tok, tag in zip(tokens[0], tags[0])}  

In [20]:
df_train['ner'] = df_train['sentences'].apply(for_bert)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [22]:
df_train.ner

0    {'Вот': 'O', 'принесли': 'O', 'эти': 'O', 'зад...
1    {'Когда': 'O', '-': 'O', 'то': 'O', 'у': 'O', ...
2    {'На': 'O', 'зоне': 'O', 'не': 'O', 'нужно': '...
3    {'Только': 'O', 'я': 'O', 'сказал': 'O', ',': ...
4    {'Мы': 'O', 'садимся': 'O', ',': 'O', 'я': 'O'...
Name: ner, dtype: object

In [ ]:
df['ner'] = df['sentences'].apply(for_bert)

In [ ]:
df.tail()

In [ ]:
df.to_pickle('nerpkl')

In [17]:
tokens, tags = ner(['Вот принесли эти задания в колледж, надо решить химию, за 11 класс и начало института.'])
for tok, tag in zip(tokens[0], tags[0]):
    print(f'{tok}\t{tag}')

Вот	O
принесли	O
эти	O
задания	O
в	O
колледж	O
,	O
надо	O
решить	O
химию	O
,	O
за	O
11	O
класс	O
и	O
начало	O
института	O
.	O


In [12]:
df_train['tokens'][0]

Вот принесли эти задания в колледж, надо решить химию, за 11 класс и начало института.